In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('SVMDATA.csv')

df.fillna(0, inplace=True)

df.head()
df.size

80396

In [3]:
df.head()

,Label,frameTime,pcm_RMSenergy_sma_max,pcm_RMSenergy_sma_min,pcm_RMSenergy_sma_range,pcm_RMSenergy_sma_maxPos,pcm_RMSenergy_sma_minPos,pcm_RMSenergy_sma_amean,pcm_RMSenergy_sma_linregc1,pcm_RMSenergy_sma_linregc2,...,NAQ,AQ,ClQ,OQa,QOQ,SQ1,SQ2,DH12,PSP,HRF
0,1,0,0.010374,0.000102,0.010271,82,149,0.002477,-3.260103e-06,0.002743,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.701640,0.000000,-10.397435
1,1,0,0.008003,0.000108,0.007896,88,5,0.001813,5.445886e-06,0.001394,...,0.153377,0.002528,0.415674,0.405545,0.488987,0.965098,0.532640,21.698599,0.275154,0.924383
2,1,0,0.011478,0.000100,0.011378,83,194,0.003527,-2.217385e-06,0.003756,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.301822,0.000000,-8.691624
3,1,0,0.005419,0.000100,0.005319,109,206,0.001289,4.164633e-07,0.001246,...,0.077525,0.001359,0.344136,0.220149,0.140251,0.980445,0.671318,19.577620,0.350059,6.190404
4,1,0,0.011950,0.000100,0.011850,79,31,0.002273,9.087521e-06,0.001574,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.080114,0.000000,-9.621048


In [4]:
columns = df.columns.tolist()

features = [f for f in filter(lambda x: x not in ['Label', 'frameTime'], columns)]
target = ['Label']

feature_df = df[features]

X = np.asarray(feature_df)

y = np.asarray(df['Label'])


In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)

X = scaler.transform(X)

In [6]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(feature_df, df['Label'], test_size=0.1, random_state=4)

X_train.shape

(181, 396)

# SequentialFeatureSelector

<img src="sequentialModelDiagram.png">

In [76]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.svm import SVC 
classifier = SVC(gamma='auto', C=10)
# Sequential Forward Selection(sfs)
feature_selector = SFS(SVC(kernel='rbf', C=1000, gamma=0.1),
           k_features=20,
           forward=True,
           verbose=2,
           scoring='roc_auc',
           cv=4)

In [80]:
from sklearn import svm
# X_train, X_test, y_train, y_test = train_test_split(feature_df, df['Label'], test_size=0.1, random_state=4)

classifier = svm.SVC(kernel='rbf', C=1000, gamma=0.1)

classifier.fit(X_train[filtered_features], y_train)

y_predict = classifier.predict(X_test[filtered_features])

In [81]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           1       1.00      0.62      0.77         8
           2       0.81      1.00      0.90        13

   micro avg       0.86      0.86      0.86        21
   macro avg       0.91      0.81      0.83        21
weighted avg       0.88      0.86      0.85        21



In [82]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.8571428571428571

In [77]:
feature_selector.fit(X_train, y_train)
feature_selector.k_feature_idx_  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 396 out of 396 | elapsed:    8.5s finished

[2020-10-04 23:43:19] Features: 1/20 -- score: 1.0[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 395 out of 395 | elapsed:    8.6s finished

[2020-10-04 23:43:27] Features: 2/20 -- score: 1.0[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 394 out of 394 | elapsed:    8.2s finished

[2020-10-04 23:43:36] Features: 3/20 -- score: 1.0[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:  

(1,
 6,
 7,
 8,
 66,
 162,
 174,
 181,
 184,
 197,
 198,
 199,
 200,
 201,
 210,
 222,
 270,
 306,
 318,
 342)

In [78]:
# features = df[features].fit(np.asarray(x_train), y_train)

In [79]:
filtered_features= df[features].columns[list(feature_selector.k_feature_idx_)]
filtered_features

Index(['pcm_RMSenergy_sma_min', 'pcm_RMSenergy_sma_linregc1',
       'pcm_RMSenergy_sma_linregc2', 'pcm_RMSenergy_sma_linregerrQ',
       'pcm_fftMag_mfcc_sma[5]_linregc1', 'pcm_zcr_sma_linregc1',
       'voiceProb_sma_linregc1', 'F0_sma_min', 'F0_sma_minPos',
       'pcm_RMSenergy_sma_de_amean', 'pcm_RMSenergy_sma_de_linregc1',
       'pcm_RMSenergy_sma_de_linregc2', 'pcm_RMSenergy_sma_de_linregerrQ',
       'pcm_RMSenergy_sma_de_stddev', 'pcm_fftMag_mfcc_sma_de[1]_linregc1',
       'pcm_fftMag_mfcc_sma_de[2]_linregc1',
       'pcm_fftMag_mfcc_sma_de[6]_linregc1',
       'pcm_fftMag_mfcc_sma_de[9]_linregc1',
       'pcm_fftMag_mfcc_sma_de[10]_linregc1',
       'pcm_fftMag_mfcc_sma_de[12]_linregc1'],
      dtype='object')